In [1]:
import torch

# Attention
There is 3 main ways to do attention: encoder-decoder attention, causal attention and bi-directional self-attention. I already implemented the first in this [repo](https://github.com/abtraore/nmt-rnn-pytorch-from-scratch). I will then focus on the later 2.